# DPO(Direct Preference Optimization)：

<span style="font-size: 20px; font-weight: bold;">注意：您使用该案例默认的数据和模型训练时，会产生一定费用。计费方式参考：https://cloud.baidu.com/doc/WENXINWORKSHOP/s/6lrk4bgxb</span>

直接偏好优化方法，通过直接优化语言模型来实现对大模型输出的精确把控，不用进行强化学习，也可以准确判断和学习到使用者的偏好，且效果更加突出。

千帆Python SDK 除了支持`Finetune`和`PostPretrain`之外也支持了`DPO`训练以进行直接的强化学习。


本例将基于qianfan==0.3.10展示通过Dataset加载本地数据集，并上传到千帆平台，基于`ERNIE-Tiny-8K`进行DPO训练，并使用Model进行批量跑评估数据，直到最终完成服务发布，并最终实现服务调用的完整过程。

In [ ]:
! pip install "qianfan[dataset_base]" -U

In [1]:
import qianfan
qianfan.__version__


'0.3.10'

## 前置准备
- 初始化千帆安全认证AK、SK

In [1]:
import os 

os.environ["QIANFAN_ACCESS_KEY"] = "your_ak"
os.environ["QIANFAN_SECRET_KEY"] = "your_sk"

#### 导入依赖
- `qianfan.trainer.consts` trainer使用中所用到的常量
- `qianfan.resources.console.consts` api层面定义的字段常量
- `qianfan.trainer.configs` trainer使用所需要的config配置数据类
- `qianfan.resources.QfMessages` 用于组装qianfan.ChatCompletion的输入messages
- `qianfan.trainer.LLMFinetune` 大语言模型dpo任务Trainer实现
- `qianfan.dataset.Dataset` 千帆dataset类，用于管理千帆平台、本地、第三方数据集的导入导出，数据清洗等操作

In [2]:
from qianfan.trainer.consts import ActionState
from qianfan.model.consts import ServiceType
from qianfan.resources.console import consts as console_consts
from qianfan.trainer.configs import TrainConfig
from qianfan.model.configs import DeployConfig
from qianfan.resources import QfMessages
from qianfan.trainer import DPO
from qianfan.dataset import Dataset
from qianfan.utils import enable_log
import logging

enable_log(logging.INFO)

## 数据集加载

千帆SDK提供了数据集实现帮助我们可以快速的加载本地的数据集到内存，并通过设定DataSource数据源以保存至本地和千帆平台。

In [8]:
from qianfan.dataset import Dataset

# 加载本地数据集
ds: Dataset = Dataset.load(data_file="./data/dpo_multi_train_template.jsonl")
ds.list()

[INFO] [04-26 18:45:15] dataset.py:408 [t:8423316288]: no data source was provided, construct
[INFO] [04-26 18:45:15] dataset.py:276 [t:8423316288]: construct a file data source from path: ./data/dpo_multi_train_template.jsonl, with args: {}
[INFO] [04-26 18:45:15] file.py:298 [t:8423316288]: use format type FormatType.Jsonl
[INFO] [04-26 18:45:15] utils.py:416 [t:8423316288]: need create cached arrow file for /Users/zhonghanjun/pywp/bce-qianfan-sdk/cookbook/finetune/data/dpo_multi_train_template.jsonl
[INFO] [04-26 18:45:15] utils.py:461 [t:8423316288]: start to write arrow table to /Users/zhonghanjun/.qianfan_cache/dataset/Users/zhonghanjun/pywp/bce-qianfan-sdk/cookbook/finetune/data/dpo_multi_train_template.arrow
[INFO] [04-26 18:45:15] utils.py:473 [t:8423316288]: writing succeeded
[INFO] [04-26 18:45:15] utils.py:347 [t:8423316288]: start to get memory_map from /Users/zhonghanjun/.qianfan_cache/dataset/Users/zhonghanjun/pywp/bce-qianfan-sdk/cookbook/finetune/data/dpo_multi_train_t

[[{'assistant': '好的，给您生成一些标题：1) 人工智能的未来与挑战； 2) 机器学习在现代生活中的应用； 3) 从AI发展历程看未来发展趋势。',
   'chosen': None,
   'rejected': None,
   'user': '生成一个关于人工智能的标题吧。'},
  {'assistant': '当然，人工智能正在各个行业带来深远影响。以下是一些具体的例子：\n1. ** 医疗保健： ** AI用于医学影像分析， 帮助医生快速诊断疾病。 它还可以预测疾病风险， 个性化治疗方案， 并改善医疗保健的效率和精确度。\n2. ** 金融服务： ** 人工智能被用于制定风险模型， 预测市场走势， 进行高频交易， 以及提供智能投资建议。\n3. ** 制造业： ** 自动化和智能机器人改变了生产线， 提高了生产效率和产品质量。 AI还用于预测维护， 减少设备损坏和停机时间。\n4. ** 教育： ** AI个性化学习， 根据学生的学习风格和进度制定课程。 它还能提供定制化的教学资源和智能辅导。\n5. ** 交通运输： ** 自动驾驶汽车利用人工智能技术， 提高了道路安全， 并在未来可能改变整个交通系统。\n这些是人工智能在各个领域带来革命性变化的一些例子， 它们正在改变我们的生活方式， 提高效率， 并为未来开辟新的可能性。',
   'chosen': None,
   'rejected': None,
   'user': '可以再给我一些具体的例子吗？'},
  {'assistant': None,
   'chosen': '是的',
   'rejected': '不是的',
   'user': '会导致这些行业大量失业么？'}],
 [{'assistant': '好的，给您生成一些标题：1) 人工智能的未来与挑战； 2) 机器学习在现代生活中的应用； 3) 从AI发展历程看未来发展趋势。',
   'chosen': None,
   'rejected': None,
   'user': '生成一个关于人工智能的标题吧。'},
  {'assistant': '当然，人工智能正在各个行业带来深远影响。以下是一些具体的例子：\n1. ** 医疗保健： ** AI用于医学影像分析， 帮助医生快速诊断疾病。 它还可以

从本地数据集上传到BOS

In [9]:
# 保存到千帆平台
from qianfan.dataset.data_source import QianfanDataSource
from qianfan.resources.console import consts as console_consts

bos_bucket_name = "bucket_name"
bos_bucket_file_path = "/sdk_ds_dpo/"
qianfan_dataset_name = "random_sdk_trainer_ds_dpo"

# 创建千帆数据集，并上传保存
qianfan_data_source = QianfanDataSource.create_bare_dataset(
    name=qianfan_dataset_name,
    template_type=console_consts.DataTemplateType.PromptChosenRejected,
    storage_type=console_consts.DataStorageType.PrivateBos,
    storage_id=bos_bucket_name,
    storage_path=bos_bucket_file_path,
)

ds = ds.save(qianfan_data_source)

[INFO] [04-26 18:45:18] baidu_qianfan.py:455 [t:8423316288]: start to create dataset on qianfan
[INFO] [04-26 18:45:18] baidu_qianfan.py:473 [t:8423316288]: create dataset on qianfan successfully
[INFO] [04-26 18:45:18] dataset.py:994 [t:8423316288]: list local dataset data by slice(0, 9999, None)
[INFO] [04-26 18:45:18] baidu_qianfan.py:241 [t:8423316288]: start to upload data to user BOS
[INFO] [04-26 18:45:18] baidu_qianfan.py:250 [t:8423316288]: upload dataset file /Users/zhonghanjun/.qianfan_cache/dataset/.qianfan_download_cache/dg-cf8kwyyjstvq9gyy/ds-xzbihgqvagt7titz/1/data_98dba347-1337-4a22-9c3b-e74de0222b84.jsonl to /sdk_ds_dpo/data_98dba347-1337-4a22-9c3b-e74de0222b84.jsonl
[INFO] [04-26 18:45:19] baidu_qianfan.py:253 [t:8423316288]: uploading data to user BOS finished
[INFO] [04-26 18:45:20] utils.py:569 [t:8423316288]: successfully create importing task
[INFO] [04-26 18:45:22] utils.py:572 [t:8423316288]: polling import task status
[INFO] [04-26 18:45:22] utils.py:579 [t:84

### DPO 训练
`DPO` 实现了DPO逻辑的trainer，它内部组装了SFT所需要的基本`Pipeline`, 用于串联数据->训练->模型发布->服务调用等步骤

In [ ]:
from qianfan.trainer.consts import PeftType

trainer = DPO(
    train_type="ERNIE-Tiny-8K",
    train_config=TrainConfig(
        epoch=1,
        learning_rate=0.000001,
        max_seq_len=4096,
        peft_type=PeftType.ALL,
    ),
    dataset=ds,
)

### 运行任务
同步运行trainer，训练直到模型发布完成

In [11]:
trainer.run()

[INFO] [04-26 18:45:47] utils.py:768 [t:6272888832]: data releasing, keep polling
[INFO] [04-26 18:45:50] utils.py:768 [t:6272888832]: data releasing, keep polling
[INFO] [04-26 18:45:52] utils.py:768 [t:6272888832]: data releasing, keep polling
[INFO] [04-26 18:45:55] utils.py:768 [t:6272888832]: data releasing, keep polling
[INFO] [04-26 18:45:57] utils.py:768 [t:6272888832]: data releasing, keep polling
[INFO] [04-26 18:46:00] utils.py:775 [t:6272888832]: data releasing succeeded
[INFO] [04-26 18:46:03] actions.py:644 [t:6272888832]: [train_action] training ... job_name:job_nVDuV5uzaT current status: Running, 1% check train task log in https://console.bce.baidu.com/qianfan/train/postPretrain/job-kpe3x2y674zz/task-9y230h64gksh/detail/traininglog
[INFO] [04-26 18:46:33] actions.py:644 [t:6272888832]: [train_action] training ... job_name:job_nVDuV5uzaT current status: Running, 1% check train task log in https://console.bce.baidu.com/qianfan/train/postPretrain/job-kpe3x2y674zz/task-9y23

获取finetune任务输出：

In [12]:
trainer.output

{'datasets': {'sourceType': 'Platform',
  'versions': [{'versionId': 'ds-xzbihgqvagt7titz'}],
  'splitRatio': 20},
 'task_id': 'task-9y230h64gksh',
 'job_id': 'job-kpe3x2y674zz',
 'metrics': {'BLEU-4': '0.17%',
  'ROUGE-1': '3.97%',
  'ROUGE-2': '0.00%',
  'ROUGE-L': '2.12%'},
 'checkpoints': [],
 'model_id': 'am-1fw9rd5r8mkc',
 'model_version_id': 'amv-s55pxvh2yy5w',
 'model': <qianfan.model.model.Model at 0x1376f6f50>}